In [1]:
import sys; sys.path.insert(0, '..')
from greenflow.dataframe_flow import TaskGraph

In [2]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38365 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 270.39 GB


We can open the status page in the brwoser by following javascript commands

In [3]:
from IPython.display import HTML
javascript = """
<script type="text/Javascript">
    function check_status(){
        var url = document.location.href;
        var index = url.indexOf(':8888');
        var status = url.substr(0, index)+":8787";
        window.open(status,'_blank');
    }
    check_status();
</script>
"""
HTML(javascript)

# Prepare distributed CSV files <span style="color:red">(not necessary if the dataset is already prepared)</span>

Following is the code to prepare the dataset as mulitple csv files that we can use dask_cuda to load them

First use this simple taskgraph to load data then sort it by the asset id and datatime

In [4]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/sort_stocks.gq.yaml')
input_cached, = task_graph.run()
task_graph.draw()

GreenflowWidget(sub=HBox(), value=[OrderedDict([('id', 'stock_data'), ('type', 'CsvStockLoader'), ('conf', {'f…

convert the sorted stock data into partitions and save it into csv files. Note, the data is slited in a way that the same asset belongs to the same partition

In [7]:
import dask.dataframe as dd
import os
num_partitions = 8

dd.from_pandas(input_cached, npartitions=num_partitions).shuffle(on=['asset']).to_csv('many-small/*.csv', index=False)

['/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/0.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/1.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/2.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/3.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/4.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/5.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/6.csv',
 '/home/yidong/Projects/gQuant/gQuant/plugins/gquant_plugin/notebooks/many-small/7.csv']

In [8]:
!cat ../taskgraphs/dask_tutorial.gq.yaml

- conf:
    file: notebooks/data/stock_price_hist.csv.gz
    path: notebooks/many-small
  id: stock_data
  inputs: {}
  module: greenflow_gquant_plugin.dataloader
  type: CsvStockLoader
- conf:
    keys:
    - asset
    - datetime
  id: sort_node
  inputs:
    in: stock_data.dask_cudf_out
  module: greenflow_gquant_plugin.transform
  type: SortNode
- conf: {}
  id: ''
  inputs:
    in1: output_csv.df_out
  type: Output_Collector
- conf:
    column: volume
  id: average_volume
  inputs:
    stock_in: sort_node.out
  module: greenflow_gquant_plugin.transform
  type: AverageNode
- conf:
    path: notebooks/dask_average_volume.csv
  id: output_csv
  inputs:
    df_in: average_volume.stock_out
  module: greenflow_gquant_plugin.analysis
  type: OutCsvNode


In [9]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/dask_tutorial.gq.yaml')
task_graph.draw()

GreenflowWidget(sub=HBox(), value=[OrderedDict([('id', 'stock_data'), ('type', 'CsvStockLoader'), ('conf', {'f…

In [10]:
task_graph.run(formated=True, profile=True)

id:stock_data process time:0.023s
id:average_volume process time:0.018s
id:output_csv process time:2.049s


In [12]:
task_graph.run()[0].compute()

,asset,volume
0,703638,274.248038
1,25156,154.621564
2,23316,859.237660
3,705451,92.104888
4,6405,38.077584
...,...,...
5047,4716,6.122072
5048,680831,96.094879
5049,708156,2017.879479
5050,24991,27.420712


clean up the generated the csv file

In [13]:
!rm dask_average_volume.csv

In [14]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}